In [ ]:
from entity_mapper import EntityMapper
from topology_dict_builder import TopologyDictBuilder
#from geometry_dict_builder import GeometryDictBuilder
from utils import load_dictionary_from_file, write_dictionary_to_file
import glob
from pathlib import Path
import os
import meshplot as mp
import numpy as np

from yaml import CLoader
import yaml
from tqdm.auto import tqdm


def process(data_path="./results_abc_00", data_format="yaml"):
    data_dir = Path(data_path)
    
    # Glob files in data dir
    geo_files = sorted(data_dir.glob(f"*_geo.{data_format}"))
    topo_files = sorted(data_dir.glob(f"*_topo.{data_format}"))
    print(len(geo_files), len(topo_files))
    for idx, g in enumerate(geo_files[:]):
        #idi = g.stem.split("_")[0]
        idi = str(g).replace("_geo", "_topo")
        if idi != str(topo_files[idx]):
            print(idi,  topo_files[idx])
        #print(idi)
    assert len(geo_files) == len(topo_files)
    
    #print(geo_files[8229])
    #return
    loops_closed = [79550, 1778290]#[0, 0]
    
    # Process files
    pbar = tqdm(range(8229, len(geo_files)))
    for i in pbar:
        pbar.set_description("Processing %s"%topo_files[i])
        topo = load_dictionary_from_file(topo_files[i], data_format)
        geo = load_dictionary_from_file(geo_files[i], data_format)
        assert geo_files[i].stem.split("_")[0] == topo_files[i].stem.split("_")[0]
        
        loops_closed_topo = parse_topo(topo, geo, -1, geo_files[i])
        loops_closed[0] += loops_closed_topo[0]
        loops_closed[1] += loops_closed_topo[1]
        #if loops_closed / 100 
        print(loops_closed)#[0], loops_closed[1]*100)
        
        write_dictionary_to_file(topo_files[i], topo, data_format)
        

process(data_format='yaml')

In [134]:
import glob, os
from entity_mapper import EntityMapper
from topology_dict_builder import TopologyDictBuilder
#from geometry_dict_builder import GeometryDictBuilder
from utils import load_dictionary_from_file, write_dictionary_to_file
import glob
from pathlib import Path
import os
import meshplot as mp
import numpy as np
import igl

from yaml import CLoader
import yaml
from tqdm.auto import tqdm


# Read in open loop information
open_loops = sorted(glob.glob("data/open_loops/out_check_1/out_check/*/*.ori"))
files = sorted(os.listdir("data/open_loops/out_check_1/out_check"))
patches = {}
for o in open_loops[:]:
    file = o.split("/")[-2]
    patches[file] = []
    with open(o, "r") as fi:
        lines = fi.readlines()
        for l in lines:
            part, patch = l.strip().split(",")
            patches[file].append((int(part), int(patch)))


def process(files, patches, data_path="./results_fixed/", data_format="yaml"):
    data_dir = Path(data_path)
    
    # Glob files in data dir
    geo_files = sorted(data_dir.glob(f"*_geo.{data_format}"))
    topo_files = sorted(data_dir.glob(f"*_topo.{data_format}"))
    assert len(geo_files) == len(topo_files)
    
    # Process files
    pbar = tqdm(range(14, 15))#len(geo_files)))
    for i in pbar:
        nr = topo_files[i].stem.split("_")[0]
        if nr not in files:
            continue

        #print(nr, patches[nr])
        pbar.set_description("Processing %s"%topo_files[i])
        topo = load_dictionary_from_file(topo_files[i], data_format)
        geo = load_dictionary_from_file(geo_files[i], data_format)
        assert geo_files[i].stem.split("_")[0] == topo_files[i].stem.split("_")[0]
        
        
        loops_closed_topo, loops_open = parse_topo(topo, geo, -1, geo_files[i])  
        print(nr, patches[nr])
        for loop in loops_open:
            if loop[0] != 2:
                continue
            print(loop[0], loop[1])
            verts, faces, curves = loop[2], loop[3], loop[4]
            p = mp.plot(verts, faces, return_plot=True)
            scale3d = (np.max(verts)-np.min(verts))/40.0
            
            points = []
            colors = []
            for cidx, c in enumerate(curves):
                if cidx < len(curves)-1:
                    print("Matching curve end ", curves[cidx][-1], curves[cidx+1][0], np.allclose(curves[cidx][-1], curves[cidx+1][0], atol=1e-4))
                #lines_s = c
                #lines_e = np.roll(c, -1, axis=0)
                #p.add_lines(lines_s[:-1], lines_e[:-1])#, shading={"line_color": color})
                #colors = np.array([cidx]*len(c))
                colors.extend(np.linspace(0.0, 1.0, len(c)))
                points.extend(c)
            print("Matching wire end ", curves[0][0], curves[-1][-1], np.allclose(curves[0][0], curves[-1][-1], atol=1e-4))

            #p.add_points(np.array(points), c=np.linspace(0., 1.0, len(points)), shading={"point_size": scale3d})
            p.add_points(np.array(points), c=np.array(colors), shading={"point_size": scale3d})


#             print("Matching wire end ", curves[0][0], curves[-1][-1], np.allclose(curves[0][0], curves[-1][-1], atol=1e-4))

#             @mp.interact(curve_idx=list(range(len(curves))))
#             def c_curve(curve_idx):
#                 global last_idi
#                 #p = mp.plot(verts, faces)
#                 idi = p.add_points(curves[curve_idx], c = np.linspace(0, 1.0, len(curves[curve_idx])), shading={"point_size": scale3d})
#     #                 if last_idi:
#     #                     p.remove_object(last_idi)
#     #                     last_idi = idi
#     #                 else:
#     #                     last_idi = idi
#             #break

                #p.add_points(np.array(points), c=np.linspace(0., 1.0, len(points)), shading={"point_size": scale3d})
        
last_idi = None
process(files, patches, data_format='yaml')

Not converted results_fixed/00000014_5b1c2f8a8c6f40fdaae1e69d_step_000_geo.yaml 4 Other
Not converted results_fixed/00000014_5b1c2f8a8c6f40fdaae1e69d_step_000_geo.yaml 4 Other
Not converted results_fixed/00000014_5b1c2f8a8c6f40fdaae1e69d_step_000_geo.yaml 5 Other
Not converted results_fixed/00000014_5b1c2f8a8c6f40fdaae1e69d_step_000_geo.yaml 5 Other
00000014 [(2, 3), (2, 12)]
2 0


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000165…

Matching curve end  [-5.745062   97.53543763  0.        ] [-5.74506195 97.53543763  0.        ] True
Matching curve end  [-5.77022052 96.77383692  0.        ] [-5.7702205  96.77383692  0.        ] True
Matching curve end  [-5.7702205  96.77383692  0.        ] [-5.77022052 96.77383692  0.        ] True
Matching wire end  [-5.745062   97.53543763  0.        ] [-5.74506195 97.53543763  0.        ] True
2 1


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000407…

Matching curve end  [-5.53659530e+00  9.52500000e+01  6.78037371e-16] [-5.5365953 95.25       0.       ] True
Matching curve end  [-5.79059244 95.50520592  0.        ] [-5.7905924  95.50520592  0.        ] True
Matching curve end  [-5.7905924  95.50520592  0.        ] [-5.79059244 95.50520592  0.        ] True
Matching curve end  [-5.5365953 95.25       0.       ] [-5.53659530e+00  9.52500000e+01  6.78037371e-16] True
Matching wire end  [ 5.53659530e+00  9.52500000e+01 -1.35607474e-15] [ 5.5365953 95.25       0.       ] True
2 2


...

Matching curve end  [-5.7234277  98.04297591  0.        ] [-5.72342770e+00  9.80429759e+01  2.10275322e-15] True
Matching curve end  [-5.74506200e+00  9.75354376e+01  2.11070154e-15] [-5.745062   97.53543763  0.        ] True
Matching curve end  [-5.745062   97.53543763  0.        ] [-5.74506200e+00  9.75354376e+01  2.11070154e-15] True
Matching wire end  [-5.7234277  98.04297591  0.        ] [-5.72342770e+00  9.80429759e+01  2.10275322e-15] True
2 11


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.859664…

Matching curve end  [-5.6545636  99.31110886  0.        ] [-5.65456360e+00  9.93111089e+01  2.07745296e-15] True
Matching curve end  [-5.68443140e+00  9.88039882e+01  2.08842622e-15] [-5.6844314  98.80398817  0.        ] True
Matching curve end  [-5.6844314  98.80398817  0.        ] [-5.68443140e+00  9.88039882e+01  2.08842622e-15] True
Matching wire end  [-5.6545636  99.31110886  0.        ] [-5.65456360e+00  9.93111089e+01  2.07745296e-15] True
2 12


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.004072…

Matching curve end  [-5.68583956e+00  1.16841820e+02  2.52502000e-14] [ -5.68583956 116.84181959   0.        ] True
Matching curve end  [ -4.52120782 120.37059058   0.        ] [ 4.52120000e+00  1.20370600e+02 -8.57300342e-15] False
Matching curve end  [ 4.52120000e+00  1.20370600e+02 -7.68829445e-15] [ 4.52120000e+00  1.20370600e+02 -8.57300342e-15] True
Matching curve end  [ 4.52120000e+00  1.20370600e+02 -7.68829445e-15] [ -4.52120782 120.37059058   0.        ] False
Matching wire end  [-5.68583956e+00  1.16841820e+02  2.52502000e-14] [ -5.68583956 116.84181959   0.        ] True



In [3]:
"2d open wires: 103742/2214383"

'2d open wires: 103742/2214383'

In [70]:
string = ""
ids = set()
for l in skippers:
    if l != "\n":
        string += l
    else:
        #print(string)
        ssplit = string.split(" ")
        if len(ssplit) >= 2:
            file = ssplit[2]
            file = file.replace("_geo.yaml", "_mesh")
            ids.add(file)
            #print(file)
        string = ""

for fi in ids:
    os.rename(fi, fi.replace("results_abc0/", "results_abc0/conv_errors"))

In [133]:
def parse_topo(topo, geo, index=-1, filename=None):
    loops_closed = [0, 0]
    if index == -1:
        start_index = 0
        end_index = None
    else:
        start_index = index
        end_index = index+10
        
    open_loops = []
        
    # Iterate over parts
    pbar_parts = tqdm(list(topo["parts"]), disable=True)
    for pi, part in enumerate(pbar_parts):
        pbar_parts.set_description("Part %i/%i\r"%(pi, len(topo["parts"])))
        
        # Skip regions for now
        
        skippers = []
        
        # Iterate over shells
        pbar_shells = tqdm(list(part["shells"]), disable=True)
        for si, shell in enumerate(pbar_shells):
            pbar_shells.set_description("Shell %i/%i\r"%(si, len(part["shells"])))
            shell_orientation = shell["orientation_wrt_region"]
            
            # Iterate over faces
            #p = mp.plot(np.array([]), np.array([]), return_plot=True)
            
            for fi, shell_face in enumerate(shell["faces"][start_index:end_index]):
                #print("Face %i/%i\r"%(fi, len(shell["faces"])))
                face_orientation = shell_face["face_orientation_wrt_shell"]
                face_index = shell_face["face_index"]
                face = part["faces"][face_index]
                
                # Get surface
                surface_index = face["surface"]
                surface = geo["parts"][pi]["surfaces"][surface_index]
                surface_orientation = face["surface_orientation"]
                
                if surface["type"] != "BSpline":
                    print("Not converted", filename, pi, surface["type"])
                    skippers.append([filename, pi])
                    break
                
                #visualize_3dsurface(surface, None)              
                
                p = None
                # Get loops
                for li in face["loops"]:
                    loop = part["loops"][li]
                    
                    curves = []
                    
                    loops_closed[1] += 1
#                    curves2d = []
                    start_point = end_point = None
                    wire_closed = True
                    for hei in loop["halfedges"]:
                        halfedge = part["halfedges"][hei]
                        he_orientation = halfedge["orientation_wrt_edge"]
                        edge = part["edges"][halfedge["edge"]]
                        curve3d = geo["parts"][pi]["3dcurves"][edge["3dcurve"]]
                        curve2d = geo["parts"][pi]["2dcurves"][halfedge["2dcurve"]]
                        
                        if curve3d["type"] not in ["BSpline"]: #or curve2d["type"] != "BSpline":
                            print("Not converted", filename, pi, curve3d["type"])
                            skippers.append([filename, pi])
                            if curve3d["type"] != "Other":
                                break
                            else:
                                continue
                        
                        #points2d = evaluate_2dcurve(curve2d, he_orientation) #2d check
                        points2d = evaluate_2dcurve(curve3d, he_orientation, surface_orientation)
                        curves.append(points2d)
                        
                        if type(start_point) == type(None):
                            start_point = points2d[0]
                            
                        if type(end_point) != type(None):
                            #print("Matching curve end ", points2d[0], end_point, np.allclose(points2d[0], end_point, atol=1e-4))
                            wire_closed = wire_closed and np.allclose(points2d[0], end_point, atol=1e-2)
                        
                        end_point = points2d[-1]
                    

                        #p = visualize_2dcurve(curve2d, he_orientation, p)

                    if type(start_point) != type(None) and type(end_point) != type(None):
                        #print("Matching wire end ", end_point, start_point, np.allclose(start_point, end_point, atol=1e-4))
                        wire_closed = wire_closed and np.allclose(start_point, end_point, atol=1e-2)
                        #print("Wire of %i curves closed, %s"%(len(loop["halfedges"]),wire_closed))
                        loop["closed"] = wire_closed
                        if not wire_closed:
                            #print("Wire of %i curves closed, %s"%(len(loop["halfedges"]),wire_closed))
                            loops_closed[0] += 1
                            
                            
                    #print(filename, type(filename))
                    obj = str(filename).replace("_geo.yaml", "_mesh")
                    obj = obj.replace("results_fixed", "results_abc_00")
                    obj += "/%03i_%05i_mesh.obj"%(pi, fi)
                    if not os.path.exists(obj):
                        print("File not existing")
                    else:
                        verts, faces = igl.read_triangle_mesh(obj)

                    open_loops.append((pi, fi, verts, faces, curves))

                                
#                                 p = mp.plot(verts, faces, return_plot=True)
#                                 scale3d = (np.max(verts)-np.min(verts))/40.0
                                
#                                 #colors = []
#                                 points = []
#                                 for cidx, c in enumerate(curves):
#                                     if cidx < len(curves)-1:
#                                         print("Matching curve end ", curves[cidx][-1], curves[cidx+1][0], np.allclose(curves[cidx][-1], curves[cidx+1][0], atol=1e-4))
#                                     #lines_s = c
#                                     #lines_e = np.roll(c, -1, axis=0)
#                                     #p.add_lines(lines_s[:-1], lines_e[:-1])#, shading={"line_color": color})
#                                     #colors = np.array([cidx]*len(c))
#                                     points.extend(c)
#                                 print("Matching wire end ", curves[0][0], curves[-1][-1], np.allclose(curves[0][0], curves[-1][-1], atol=1e-4))

#                                 #p.add_points(np.array(points), c=np.linspace(0., 1.0, len(points)), shading={"point_size": scale3d})
                        #print(li, wire_closed)
                        
                #halfedges = loops["halfedges"]
                
    return loops_closed, open_loops
                
                    
from geomdl import BSpline, NURBS, trimming

def evaluate_2dcurve(curve, orientation, surface_orientation):
    crv2d = NURBS.Curve()
    crv2d.degree = curve["degree"]
    crv2d.ctrlpts = curve["poles"]
    crv2d.knotvector = curve["knots"]
    crv2d.weights = curve["weights"]
    #print(orientation, surface_orientation, orientation != surface_orientation)
    ori = not orientation #!= surface_orientation
    crv2d.opt = ['reversed', ori]
    
    
    
    ps2d = crv2d.evalpts
    if crv2d.opt["reversed"]:
        ps2d.reverse()
    return np.array(ps2d)
        
#     if not p:
#         p = mp.plot(, return_plot=True)
#     else:
#         p.add_points(np.array(ps2d))
        
#     return p


# vertices = [
#     -10.0, -15.0, 0.0,
#     -10.0,   5.0, 0.0,
#      10.0,   5.0, 0.0,
#     -10.0, -15.0, 0.0,
#      10.0, -15.0, 0.0,
#     -10.0,   5.0, 0.0,
#       0.0,  15.0, 0.0,
#      10.0,   5.0, 0.0,
#      10.0, -15.0, 0.0
# ]

# colors = [
#     0xff,
#     0xffff,
#     0xff00ff,
#     0x00ffff,
#     0xffff00,
#     0xff00,
#     0xff0000,
#     0xff00ff,
#     0xffffff
# ]

# line = k3d.line(vertices, colors=colors, width=0.5)

# plot = k3d.plot(antialias=True)
# plot += line
# plot.display()


def visualize_3dsurface(surf, curves3d, plot):
    srv3d = NURBS.Surface()
    srv3d.degree_u = surf["u_degree"]
    srv3d.degree_v = surf["v_degree"]

    srv3d.ctrlpts_size_u = len(surf["poles"])
    srv3d.ctrlpts_size_v = len(surf["poles"][0])
    srv3d.ctrlpts = [item for sublist in surf["poles"] for item in sublist]

    srv3d.knotvector_u = surf["u_knots"]
    srv3d.knotvector_v = surf["v_knots"]

    srv3d.weights = [item for sublist in surf["weights"] for item in sublist]

    # Set evaluation delta
    srv3d.delta = 0.05#25

    # Evaluate surface points
    srv3d.evaluate()
    ps3ds = srv3d.evalpts
    ves = np.array(srv3d.vertices)
    fes = []
    for fss in range(len(srv3d.faces)):
        fes.append(srv3d.faces[fss].vertex_ids)
        
    mp.plot(np.array(ves), np.array(fes))
#     p3d = mp.plot(np.array(verts), np.array(faces), np.array(colors), shading={"wireframe": True}, return_plot=True)
#     #p3d = mp.plot(np.array(pts3ds), shading={"point_size": 0.5}, return_plot=True)
#     #p3d = mp.plot(np.array(ves), np.array(fes), shading={"wireframe": True}, return_plot=True)
#     p3d.add_points(np.array(pts3d[::5]), c=np.array(cls[::5]), shading={"point_size": 0.5})
#     for il, cl in enumerate(lines3ds):
#         p3d.add_lines(lines3ds[il][1:], lines3de[il][1:], shading={"line_color": cls_lines[il]})


#def visualize_2dsurface(loops, trim_doman, face_domain, surface_orientation):
    
    
                

In [14]:
import os
import glob
from shutil import rmtree
res = glob.glob("results_abc0/*_geo.yaml")
folds_e = glob.glob("results_abc0/*_mesh")
folds_s = []
for f in res:
    fold = f.replace("_geo.yaml", "_mesh")
    if os.path.isdir(fold):
        #print(fold)
        folds_e.remove(fold)
    #print(f)
for fold in folds_e:
    rmtree(fold)
print(folds_e)
#for 

['results_abc0/00001572_e962204720bd4897a379cb67_step_002_mesh', 'results_abc0/00001022_b09949bbddbc473d9152195d_step_000_mesh', 'results_abc0/00000773_7db0db3f25e44340bc50e750_step_000_mesh', 'results_abc0/00002295_6b1946635efe4ef6b63e83d8_step_000_mesh', 'results_abc0/00001756_c96884d40e944563908d7d2c_step_000_mesh', 'results_abc0/00000099_7911d7c4c4b9417d92801ff6_step_000_mesh', 'results_abc0/00000503_654cafbd215145f78dc2d5f9_step_000_mesh', 'results_abc0/00001252_96503b6a26c64074ad598927_step_014_mesh', 'results_abc0/00002045_f5f0cb8f8edf486a9023ef1d_step_019_mesh']


In [32]:
import k3d


class Viewer():

    __plot = None
    __objects = {}
    __cnt = 0
    
    def __init__(self):
        # Plot settings
        settings = {"antialias":5, "axes": ["x", "y", "z"], "axes_helper": 1.0, "background_color":0xffffff,
           "camera_auto_fit": False, "grid_auto_fit": False, "grid_visible": False, "screenshot_scale": 2.0,
           "grid": (-1, -1, -1, 1, 1, 1), "lighting": 1.5, "menu_visibility": True, "voxel_paint_color": 0,
           "camera_fov": 60.0, "time": 0.0, "name": None, "camera_mode": 'trackball', "snapshot_include_js": True,
           "auto_rendering": True, "camera_no_zoom": False, "camera_no_rotate": False, "camera_no_pan": False,
           "fps": 25.0, "height": 1000} # manipulate_mode: translate, rotate, scale 
        self.__plot = k3d.plot(**settings)
        
    def __update_view(self):
        if len(self.__objects) == 0:
            return
        ma = np.zeros((len(self.__objects), 3))
        mi = np.zeros((len(self.__objects), 3))
        for r, obj in enumerate(self.__objects):
            ma[r] = self.__objects[obj]["max"]
            mi[r] = self.__objects[obj]["min"]
        ma = np.max(ma, axis=0)
        mi = np.min(mi, axis=0)
        diag = np.linalg.norm(ma-mi)
        mean = ((ma - mi) / 2 + mi).tolist()
        scale = 1.5 * diag
        camera = [mean[0], mean[1], mean[2]+scale, mean[0], mean[1], mean[2], 0.0, 1.0, 0.0]
        self.__plot.display()
        self.__plot.camera = camera
        self.__plot.camera_mode = "orbit"
        
    def add_mesh(self, vertices, faces, colors=0xffd200):
        # Convert to proper types
        vertices = vertices.astype("float32", copy=False)
        faces = faces.astype("uint32", copy=False)
        
        # Add to plot
        mesh = k3d.mesh(vertices, faces, color=colors, side="double")
        self.__plot += mesh
        mesh_obj = {}
        mesh_obj["max"] = np.max(vertices, axis=0)
        mesh_obj["min"] = np.min(vertices, axis=0)
        mesh_obj["mesh"] = mesh
        mesh_obj["type"] = "Mesh"
        self.__objects[self.__cnt] = mesh_obj
        self.__cnt += 1
        self.__update_view()
        
    def add_vectors(self, origins, vectors, colors=None):
        origins = origins.astype("float32", copy=False)
        vectors = vectors.astype("float32", copy=False)
        vectors = k3d.vectors(origins, vectors, color=colors)#, labels=[], label_size=1.5)
        self.__plot += vectors
        #self.__update_view()

    def add_lines(self, lines, width=0.01, colors=[]):
        lines = lines.astype("float32", copy=False)
        klines = k3d.line(lines, colors=colors, width=width)#, labels=[], label_size=1.5)
        self.__plot += klines
        self.__update_view()


def plot(vertices, faces, colors=0xffd200):
    plot = Viewer()
    #plot.add_mesh(vertices, faces, colors)

    return plot

In [18]:
p = k3d.plot()
k3d.line??

Signature:
k3d.line(
    vertices,
    color=255,
    colors=[],
    attribute=[],
    color_map=array([0.00000e+00, 1.46200e-03, 4.66000e-04, ..., 9.88362e-01,
       9.98364e-01, 6.44924e-01], dtype=float32),
    color_range=[],
    width=0.01,
    shader='thick',
    radial_segments=8,
    name=None,
    compression_level=0,
    **kwargs,
)
Source:   
def line(vertices, color=_default_color, colors=[],  # lgtm [py/similar-function]
         attribute=[], color_map=default_colormap, color_range=[], width=0.01,
         shader='thick', radial_segments=8, name=None, compression_level=0, **kwargs):
    """Create a Line drawable for plotting segments and polylines.

    Arguments:
        vertices: `array_like`.
            Array with (x, y, z) coordinates of segment endpoints.
        color: `int`.
            Packed RGB color of the lines (0xff0000 is red, 0xff is blue) when `colors` is empty.
        colors: `array_like`.
            Array of int: packed RGB colors (0xff0000 is red, 0

In [9]:
15956/ 345808

0.04614121130800907